## This notebook is to select a single set of district, age group and gender, run through many hyperparameters, cluster them and select optimal scenario

### Uploading libraries

In [ ]:
import pandas as pd
import numpy as np
from prophet import Prophet
import plotly.express as px
import plotly.graph_objects as go
import logging
import warnings
import contextlib
from itertools import product
import multiprocessing
from functools import partial
import os
from sklearn.metrics import mean_squared_error

from sklearn.cluster import AgglomerativeClustering
import numpy as np
from sklearn.linear_model import LinearRegression

### Getting rid of warning and logging messages

In [ ]:
# Suppress logging messages from cmdstanpy
logger = logging.getLogger('cmdstanpy')
logger.setLevel(logging.ERROR)
for handler in logger.handlers:
    handler.setLevel(logging.ERROR)

# Suppress SettingWithCopyWarning
warnings.filterwarnings('ignore', category=pd.errors.SettingWithCopyWarning)

### Functions

In [ ]:
# Context manager to suppress logging
@contextlib.contextmanager
def suppress_logging():
    logging.disable(logging.CRITICAL)
    try:
        yield
    finally:
        logging.disable(logging.NOTSET)

# Function to load and preprocess data
def load_and_preprocess(train_file_path, test_file_path):
    train_data = pd.read_csv(train_file_path)
    test_data = pd.read_csv(test_file_path)

    train_data['as_of_date_id'] = train_data['as_of_date_id'].astype(int)
    train_data['age_bin_id'] = train_data['age_bin_id'].astype(str)
    train_data['gender_id'] = train_data['gender_id'].astype(str)
    train_data['district_id'] = train_data['district_id'].astype(str)

    test_data['as_of_date_id'] = test_data['as_of_date_id'].astype(int)
    test_data['age_bin_id'] = test_data['age_bin_id'].astype(str)
    test_data['gender_id'] = test_data['gender_id'].astype(str)
    test_data['district_id'] = test_data['district_id'].astype(str)

    for age_bin in train_data['age_bin_id'].unique():
        for gender in train_data['gender_id'].unique():
            for district in train_data['district_id'].unique():
                mask = (train_data['age_bin_id'] == age_bin) & (train_data['gender_id'] == gender) & (train_data['district_id'] == district)
                count_75 = train_data.loc[mask & (train_data['as_of_date_id'] == 75), 'count'].values
                count_77 = train_data.loc[mask & (train_data['as_of_date_id'] == 77), 'count'].values
                if len(count_75) > 0 and len(count_77) > 0:
                    avg_count = (count_75[0] + count_77[0]) / 2
                    train_data.loc[mask & (train_data['as_of_date_id'] == 76), 'count'] = avg_count

    # Filter train_data to start from as_of_date_id 70
    train_data = train_data[train_data['as_of_date_id'] >= 70].reset_index(drop=True)

    # Assume start date and convert 'as_of_date_id' to datetime
    start_date = pd.to_datetime('2000-01-01')
    train_data['ds'] = start_date + pd.to_timedelta(train_data['as_of_date_id'], unit='D')
    test_data['ds'] = start_date + pd.to_timedelta(test_data['as_of_date_id'], unit='D')

    return train_data, test_data

# Function to normalize data
def normalize_data(df, column):
    mean = df[column].mean()
    std = df[column].std()
    df[column] = (df[column] - mean) / std
    return mean, std

# Function to denormalize data
def denormalize_data(df, column, mean, std):
    df[column] = df[column] * std + mean
    return df

# Function to train a model for a specific scenario and hyperparameters
def train_model(train_data, district, age_bin, gender, hyperparams):
    mask = (train_data['district_id'] == district) & (train_data['age_bin_id'] == age_bin) & (train_data['gender_id'] == gender)
    subset_data = train_data[mask]

    # Check if subset_data has at least 2 non-NaN rows
    if len(subset_data) < 2:
        return None, None, None

    # Normalize the data
    mean, std = normalize_data(subset_data, 'count')
    
    subset_data = subset_data.rename(columns={'ds': 'ds', 'count': 'y'})
    model = Prophet(
        yearly_seasonality=hyperparams['yearly_seasonality'],
        changepoint_prior_scale=hyperparams['changepoint_prior_scale'],
        seasonality_prior_scale=hyperparams['seasonality_prior_scale'],
        changepoint_range=hyperparams['changepoint_range']
    )
    model.add_seasonality(
        name='12-period',
        period=12, 
        fourier_order=hyperparams['fourier_order']
    )

    with suppress_logging():
        model.fit(subset_data[['ds', 'y']])
    
    return model, mean, std

# Function to calculate the RMSE for the training dataset
def calculate_rmse(model, train_data, district, age_bin, gender):
    mask_train = (train_data['district_id'] == district) & (train_data['age_bin_id'] == age_bin) & (train_data['gender_id'] == gender)
    subset_train_data = train_data[mask_train]

    train_forecast = model.predict(subset_train_data[['ds']])
    rmse = np.sqrt(np.mean((train_forecast['yhat'].values - subset_train_data['count'].values)**2))
    return rmse

# Function to train and evaluate a model for a specific scenario and hyperparameters
def train_and_evaluate_model(train_data, test_data, district, age_bin, gender, hyperparam_tuple):
    hyperparams = dict(zip(hyperparameter_space.keys(), hyperparam_tuple))
    model, mean, std = train_model(train_data, district, age_bin, gender, hyperparams)
    if model is not None:
        rmse = calculate_rmse(model, train_data, district, age_bin, gender)
        return model, mean, std, rmse, hyperparams
    return None

# Function to make predictions for a specific scenario
def make_scenario_predictions(model, mean, std, test_data, district, age_bin, gender):
    mask_test = (test_data['district_id'] == district) & (test_data['age_bin_id'] == age_bin) & (test_data['gender_id'] == gender)
    subset_test_data = test_data[mask_test]

    future = subset_test_data[['ds']]
    forecast = model.predict(future)

    # Denormalize the predictions
    forecast['yhat'] = forecast['yhat'] * std + mean

    with warnings.catch_warnings():
        warnings.simplefilter('ignore', category=pd.errors.SettingWithCopyWarning)
        subset_test_data['Prediction'] = forecast['yhat'].values
        subset_test_data['Prediction'] = subset_test_data['Prediction'].iloc[::-1].values

    return subset_test_data[['ID', 'district_id', 'age_bin_id', 'gender_id', 'as_of_date_id', 'Prediction']]

def cluster_predictions(models, test_data, district, age_bin, gender, n_clusters=5):
    predictions_list = []
    model_info = []

    for model, mean, std, rmse, hyperparams in models:
        predictions = make_scenario_predictions(model, mean, std, test_data, district, age_bin, gender)
        preds = predictions['Prediction'].values
        predictions_list.append(preds)
        model_info.append((model, mean, std, rmse, hyperparams))

    predictions_array = np.array(predictions_list)

    # Perform clustering
    clustering = AgglomerativeClustering(n_clusters=n_clusters, affinity='euclidean', linkage='ward')
    labels = clustering.fit_predict(predictions_array)

    # Select one model per cluster
    unique_models = []
    for cluster_id in range(n_clusters):
        cluster_indices = np.where(labels == cluster_id)[0]
        selected_index = cluster_indices[0]  # Select the first model in each cluster
        unique_models.append(model_info[selected_index])

    return unique_models


def print_top_models_in_cluster(models, test_data, district, age_bin, gender, n_clusters=5):
    predictions_list = []
    model_info = []

    for model, mean, std, rmse, hyperparams in models:
        predictions = make_scenario_predictions(model, mean, std, test_data, district, age_bin, gender)
        preds = predictions['Prediction'].values
        predictions_list.append(preds)
        model_info.append((model, mean, std, rmse, hyperparams))

    predictions_array = np.array(predictions_list)

    # Perform clustering
    clustering = AgglomerativeClustering(n_clusters=n_clusters, affinity='euclidean', linkage='ward')
    labels = clustering.fit_predict(predictions_array)

    # Dictionary to store models in each cluster
    cluster_dict = {i: [] for i in range(n_clusters)}
    for i, label in enumerate(labels):
        cluster_dict[label].append(model_info[i])

    return cluster_dict

def display_top_models(cluster_dict, cluster_id, top_n=5):
    if cluster_id in cluster_dict:
        cluster_models = cluster_dict[cluster_id]
        # Sort models in the cluster based on RMSE
        cluster_models.sort(key=lambda x: x[3])
        top_models = cluster_models[:top_n]

        print(f"Top {top_n} models in cluster {cluster_id}:")
        for i, (model, mean, std, rmse, hyperparams) in enumerate(top_models):
            print(f"Model {i+1}: RMSE = {rmse}, Hyperparameters = {hyperparams}")
    else:
        print(f"Cluster {cluster_id} not found.")

def plot_predictions_for_selected_model(models, test_data, district, age_bin, gender, selected_hyperparams):
    # Find the model with the specified hyperparameters
    selected_model_info = None
    for model, mean, std, rmse, hyperparams in models:
        if hyperparams == selected_hyperparams:
            selected_model_info = (model, mean, std, rmse, hyperparams)
            break
    
    if selected_model_info is None:
        print("Model with the specified hyperparameters not found.")
        return
    
    model, mean, std, rmse, hyperparams = selected_model_info
    
    # Make predictions
    predictions = make_scenario_predictions(model, mean, std, test_data, district, age_bin, gender)
    
    # Add training data
    train_subset = train_data[(train_data['district_id'] == district) & 
                              (train_data['age_bin_id'] == age_bin) & 
                              (train_data['gender_id'] == gender)]
    
    # Fit a linear regression to the training data
    X_train = train_subset['as_of_date_id'].values.reshape(-1, 1)
    y_train = train_subset['count'].values
    linear_regressor = LinearRegression()
    linear_regressor.fit(X_train, y_train)
    
    # Create a range for the trend line from the start of training to the end of predictions
    start_train = train_subset['as_of_date_id'].min()
    end_pred = predictions['as_of_date_id'].max()
    X_trend = np.arange(start_train, end_pred + 1).reshape(-1, 1)
    y_trend = linear_regressor.predict(X_trend)
    
    # Create a dataframe for the trend line
    trend_df = pd.DataFrame({
        'as_of_date_id': X_trend.flatten(),
        'trend': y_trend
    })

    # Plot the predictions
    fig = go.Figure()
    
    # Add training data
    fig.add_trace(go.Scatter(
        x=train_subset['as_of_date_id'], 
        y=train_subset['count'], 
        mode='lines+markers', 
        name='Training Data',
        line=dict(color='blue')
    ))

    # Add prediction data
    fig.add_trace(go.Scatter(
        x=predictions['as_of_date_id'], 
        y=predictions['Prediction'], 
        mode='lines+markers', 
        name='Predictions',
        line=dict(color='red')
    ))

    # Add trend line
    fig.add_trace(go.Scatter(
        x=trend_df['as_of_date_id'], 
        y=trend_df['trend'], 
        mode='lines', 
        name='Trend Line',
        line=dict(color='rgba(0, 0, 0, 0.5)', width=1, dash="dash"),
        showlegend=True
    ))

    # Calculate the min and max values for y-axis
    all_y_values = pd.concat([train_subset['count'], predictions['Prediction'], trend_df['trend']])
    y_min = all_y_values.min() * 0.99
    y_max = all_y_values.max() * 1.01

    # Add vertical lines for every 12 points
    vertical_lines = range(start_train + 12, end_pred, 12)
    for period in vertical_lines:
        fig.add_shape(
            type="line",
            x0=period,
            y0=y_min,
            x1=period,
            y1=y_max,
            line=dict(color="Green", width=2, dash="dash")
        )
    
    fig.update_layout(
        title=f'Population Count per Period of Time (District: {district}, Age Bin: {age_bin}, Gender: {gender})',
        xaxis_title='Time Period',
        yaxis_title='Population Count',
        legend_title='Dataset',
        height=500
    )

    fig.show()

def display_top_hyperparams_with_extreme_sums(cluster_dict, cluster_id, top_n=3):
    if cluster_id in cluster_dict:
        cluster_models = cluster_dict[cluster_id]
        hyperparams_with_sums = []

        for model, mean, std, rmse, hyperparams in cluster_models:
            predictions = make_scenario_predictions(model, mean, std, test_data, district, age_bin, gender)
            total_sum = predictions['Prediction'].sum()
            hyperparams_with_sums.append((hyperparams, total_sum))

        # Sort by the sum of predicted values
        hyperparams_with_sums.sort(key=lambda x: x[1], reverse=True)
        
        print(f"Top {top_n} hyperparameter sets with highest sums in cluster {cluster_id}:")
        for i in range(min(top_n, len(hyperparams_with_sums))):
            print(f"Model {i+1}: Hyperparameters = {hyperparams_with_sums[i][0]}, Sum of Predictions = {hyperparams_with_sums[i][1]}")

        hyperparams_with_sums.sort(key=lambda x: x[1])
        
        print(f"\nTop {top_n} hyperparameter sets with lowest sums in cluster {cluster_id}:")
        for i in range(min(top_n, len(hyperparams_with_sums))):
            print(f"Model {i+1}: Hyperparameters = {hyperparams_with_sums[i][0]}, Sum of Predictions = {hyperparams_with_sums[i][1]}")

    else:
        print(f"Cluster {cluster_id} not found.")

### Main file to run a specific scenario through many possible hyperparameters and then cluster them to investigate specified cluster

In [ ]:
# Main execution
if __name__ == '__main__':
    train_file_path = 'data/train.csv'
    test_file_path = 'data/test.csv'

    # Load and preprocess data
    train_data, test_data = load_and_preprocess(train_file_path, test_file_path)

    # Enter the desired scenario (district, age bin, and gender)
    district = '19'
    age_bin = '3'
    gender = '0'

    # Define the hyperparameter search space
    hyperparameter_space = {
        'yearly_seasonality': ['auto'],
        'changepoint_prior_scale': [0.001, 0.01, 0.03, 0.05, 0.075, 0.1, 0.3, 0.5, 0.7, 0.9, 1.0],
        'seasonality_prior_scale': [1, 5, 10, 15, 20],
        'changepoint_range': [0.5, 0.6, 0.7, 0.8, 0.9, 0.95],
        'fourier_order': [1, 3, 5, 7, 10, 15, 20]
    }

    # Generate all possible combinations of hyperparameters
    hyperparameter_combinations = list(product(*hyperparameter_space.values()))

    # Get the total number of CPU cores
    total_cores = os.cpu_count()
    
    # Calculate the number of cores to use, leaving 5 cores spare
    num_cores = max(1, total_cores - 5)
    
    # Create a pool of worker processes with the specified number of cores
    pool = multiprocessing.Pool(processes=num_cores)

    # Create a partial function with fixed arguments
    train_and_evaluate_partial = partial(train_and_evaluate_model, train_data, test_data, district, age_bin, gender)

    # Train and evaluate models with different hyperparameter combinations in parallel
    results = pool.map(train_and_evaluate_partial, hyperparameter_combinations)

    # Close the pool of worker processes
    pool.close()
    pool.join()

    # Filter out None results and create the models list
    models = [result for result in results if result is not None]

    # Sort the models based on their performance (RMSE)
    models.sort(key=lambda x: x[3])

    # Cluster predictions to get unique models
    unique_models = cluster_predictions(models, test_data, district, age_bin, gender, n_clusters=5)

    # Plot the predictions for the unique models
    fig = go.Figure()

    # Add training data
    train_subset = train_data[(train_data['district_id'] == district) & 
                            (train_data['age_bin_id'] == age_bin) & 
                            (train_data['gender_id'] == gender)]
    fig.add_trace(go.Scatter(
        x=train_subset['as_of_date_id'], 
        y=train_subset['count'], 
        mode='lines+markers', 
        name='Training Data',
        line=dict(color='blue')
    ))

    # Fit a linear regression to the training data
    X_train = train_subset['as_of_date_id'].values.reshape(-1, 1)
    y_train = train_subset['count'].values
    linear_regressor = LinearRegression()
    linear_regressor.fit(X_train, y_train)
    
    # Create a range for the trend line from the start of training to the end of predictions
    start_train = train_subset['as_of_date_id'].min()
    end_pred = test_data['as_of_date_id'].max()
    X_trend = np.arange(start_train, end_pred + 1).reshape(-1, 1)
    y_trend = linear_regressor.predict(X_trend)
    
    # Create a dataframe for the trend line
    trend_df = pd.DataFrame({
        'as_of_date_id': X_trend.flatten(),
        'trend': y_trend
    })

    # Add prediction data for each unique model
    colors = ['red', 'green', 'orange', 'purple', 'black']
    for i, (model, mean, std, _, _) in enumerate(unique_models):
        predictions = make_scenario_predictions(model, mean, std, test_data, district, age_bin, gender)
        fig.add_trace(go.Scatter(
            x=predictions['as_of_date_id'], 
            y=predictions['Prediction'], 
            mode='lines+markers', 
            name=f'Predictions (Model {i+1})',
            line=dict(color=colors[i])
        ))

    # Add trend line
    fig.add_trace(go.Scatter(
        x=trend_df['as_of_date_id'], 
        y=trend_df['trend'], 
        mode='lines', 
        name='Trend Line',
        line=dict(color='rgba(0, 0, 0, 0.5)', width=1, dash="dash"),
        showlegend=True
    ))

    # Calculate the min and max values for y-axis
    all_y_values = pd.concat([train_subset['count']] + [make_scenario_predictions(model, mean, std, test_data, district, age_bin, gender)['Prediction'] for model, mean, std, _, _ in unique_models] + [trend_df['trend']])
    y_min = all_y_values.min() * 0.99
    y_max = all_y_values.max() * 1.01

    # Add vertical lines for every 12 points
    vertical_lines = range(start_train + 12, end_pred, 12)
    for period in vertical_lines:
        fig.add_shape(
            type="line",
            x0=period,
            y0=y_min,
            x1=period,
            y1=y_max,
            line=dict(color="Green", width=2, dash="dash")
        )
    
    fig.update_layout(
        title=f'Population Count per Period of Time (District: {district}, Age Bin: {age_bin}, Gender: {gender})',
        xaxis_title='Time Period',
        yaxis_title='Population Count',
        legend_title='Dataset',
        height=500
    )

    fig.show()

### Select cluster (model) number to get optimal hyperparameters

In [ ]:
# Cluster predictions to get unique models
cluster_dict = print_top_models_in_cluster(models, test_data, district, age_bin, gender, n_clusters=5)

# Display top models in a specific cluster
model_nr = 3
selected_cluster_id = model_nr - 1  # Change this to the desired cluster ID
# display_top_models(cluster_dict, selected_cluster_id, top_n=5)

# Display top hyperparameter sets with the highest and lowest sums of predicted values
display_top_hyperparams_with_extreme_sums(cluster_dict, selected_cluster_id, top_n=3)

### Select a set of huperparameters to plot predictions for this specific case

In [ ]:
# Define the hyperparameters of the model you want to plot
selected_hyperparams = {'yearly_seasonality': 'auto', 'changepoint_prior_scale': 0.7, 'seasonality_prior_scale': 1, 'changepoint_range': 0.95, 'fourier_order': 7}

# Call the function to plot predictions for the selected model
plot_predictions_for_selected_model(models, test_data, district, age_bin, gender, selected_hyperparams)
